<a href="https://colab.research.google.com/github/fernandoortiz5/dashboards-interativos/blob/main/doencas-cardiacas-final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet pandas plotly ipywidgets
import pandas as pd
import plotly.express as px
from ipywidgets import interact, widgets
from IPython.display import display, clear_output

print("✅ Ambiente configurado com sucesso!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.0 MB/s eta 0:00:00
✅ Ambiente configurado com sucesso!


In [2]:
def carregar_dados():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"
    colunas = [
        'idade', 'sexo', 'tipo_dor', 'pressao_repouso', 'colesterol',
        'glicemia', 'eletrocardiograma', 'freq_maxima', 'angina_induzida',
        'depressao_st', 'inclinacao_st', 'vasos_col', 'talassemia', 'diagnostico'
    ]

    df = (
        pd.read_csv(url, names=colunas, na_values='?')
        .dropna()
        .assign(
            diagnostico = lambda x: x['diagnostico'].gt(0).map({True: 'Doença', False: 'Saudável'}),
            sexo = lambda x: x['sexo'].map({0: 'Feminino', 1: 'Masculino'}),
            faixa_etaria = lambda x: pd.cut(x['idade'],
                                          bins=[20, 40, 60, 100],
                                          labels=['20-40', '41-60', '60+'])
        )
    )
    return df

df = carregar_dados()
print("\n🔍 Dataset totalmente processado:")
display(df.head(2))



🔍 Dataset totalmente processado:


,idade,sexo,tipo_dor,pressao_repouso,colesterol,glicemia,eletrocardiograma,freq_maxima,angina_induzida,depressao_st,inclinacao_st,vasos_col,talassemia,diagnostico,faixa_etaria
0,63.0,Masculino,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,Saudável,60+
1,67.0,Masculino,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,Doença,60+


In [3]:
from google.colab import output
output.enable_custom_widget_manager()

def config_viz():
    import plotly.io as pio
    pio.renderers.default = 'colab'
    pio.templates.default = 'plotly_white'

config_viz()
print("⚙️ Renderização configurada para o Colab!")


⚙️ Renderização configurada para o Colab!


In [5]:
@interact
def analise_interativa(
    faixa=widgets.Dropdown(options=df['faixa_etaria'].cat.categories.tolist(), description='Faixa Etária:'),
    sexo=widgets.SelectMultiple(options=df['sexo'].unique(), description='Sexo:', value=['Masculino'])
):
    # Aplicar filtros
    dados = df[(df['faixa_etaria'] == faixa) & (df['sexo'].isin(sexo))]

    # Layout dinâmico
    clear_output(wait=True)

    # Visualização 1: Distribuição de Diagnósticos
    fig1 = px.histogram(
        dados, x='idade', color='diagnostico',
        title=f'Distribuição por Idade - {faixa}',
        labels={'idade': 'Idade (anos)', 'count': 'Pacientes'},
        color_discrete_map={'Saudável': '#2ecc71', 'Doença': '#e74c3c'}
    )

    # Visualização 2: Relação Clínica
    fig2 = px.scatter(
        dados, x='pressao_repouso', y='colesterol',
        color='diagnostico', size='idade',
        title='Relação Pressão Arterial x Colesterol',
        labels={'pressao_repouso': 'Pressão (mmHg)', 'colesterol': 'Colesterol (mg/dl)'}
    )

    # Visualização 3: Correlações
    corr = dados[['idade', 'pressao_repouso', 'colesterol', 'freq_maxima']].corr()
    fig3 = px.imshow(
        corr.round(2), text_auto=True,
        title='Mapa de Calor de Correlações',
        color_continuous_scale='RdBu', zmin=-1, zmax=1
    )

    # Exibição organizada
    display(fig1)
    display(fig2)
    display(fig3)

    # Dados relevantes
    print(f"\n📌 Resumo Estatístico ({len(dados)} pacientes):")
    display(dados[['pressao_repouso', 'colesterol']].describe())


interactive(children=(Dropdown(description='Faixa Etária:', options=('20-40', '41-60', '60+'), value='20-40'),…